In [ ]:
def master_array(gen_array, sim_array, cut_array):
    
    id_particle = ak.to_numpy(gen_array["event"])
    id_reco = ak.to_numpy(sim_array["event"])
    id_both = ak.to_numpy(cut_array["event"])
    print("Start masking arrays")
    # Mask out overlapping events
    mask_particle_only = ~np.isin(id_particle, id_both)
    mask_reco_only = ~np.isin(id_reco, id_both)

    particle_only = gen_array[mask_particle_only]
    reco_only = sim_array[mask_reco_only]
    
    particle_only = ak.with_field(particle_only, True, "pass_particle")
    particle_only = ak.with_field(particle_only, False, "pass_reco")

    reco_only = ak.with_field(reco_only, False, "pass_particle")
    reco_only = ak.with_field(reco_only, True, "pass_reco")

    cut_array = ak.with_field(cut_array, True, "pass_particle")
    cut_array = ak.with_field(cut_array, True, "pass_reco")
    
    print("Done masking") 
    
    reco_field_order = [
    'run', 'event', 'eventWeight',
    'ptl1', 'truth_ptl1',
    'etal1', 'truth_etal1',
    'phil1', 'truth_phil1',
    'ptb1', 'truth_ptb1', 'ptb2', 'truth_ptb2',
    'ptb3', 'truth_ptb3', 'ptb4', 'truth_ptb4',
    'etab1', 'truth_etab1', 'etab2', 'truth_etab2',
    'etab3', 'truth_etab3', 'etab4', 'truth_etab4',
    'phib1', 'truth_phib1', 'phib2', 'truth_phib2',
    'phib3', 'truth_phib3', 'phib4', 'truth_phib4',
    'mb1', 'truth_mb1', 'mb2', 'truth_mb2',
    'mb3', 'truth_mb3', 'mb4', 'truth_mb4',
    'ptj1', 'truth_ptj1', 'ptj2', 'truth_ptj2',
    'ptj3', 'truth_ptj3', 'ptj4', 'truth_ptj4',
    'ptj5', 'truth_ptj5', 'ptj6', 'truth_ptj6',
    'etaj1', 'truth_etaj1', 'etaj2', 'truth_etaj2',
    'etaj3', 'truth_etaj3', 'etaj4', 'truth_etaj4',
    'etaj5', 'truth_etaj5', 'etaj6', 'truth_etaj6',
    'phij1', 'truth_phij1', 'phij2', 'truth_phij2',
    'phij3', 'truth_phij3', 'phij4', 'truth_phij4',
    'phij5', 'truth_phij5', 'phij6', 'truth_phij6',
    'mj1', 'truth_mj1', 'mj2', 'truth_mj2',
    'mj3', 'truth_mj3', 'mj4', 'truth_mj4',
    'mj5', 'truth_mj5', 'mj6', 'truth_mj6',
    'met', 'truth_met', 'metphi', 'truth_metphi',
    'ptwhad', 'truth_ptwhad', 'truth_pt_particlewhad',
    'rapiditywhad', 'truth_rapidity_particlewhad',
    'mwhad', 'truth_mwhad',
    'mbl_selected', 'truth_m_particlebl',
    'mbwhad_selected', 'truth_m_particlebwhad',
    'dRbl_selected', 'truth_dR_particlebl',
    'dRbwhad_selected', 'truth_dR_particlebwhad',
    'pass_particle', 'pass_reco'
    ]

    # Fill missing fields with None and keep only listed fields in specified order
    def harmonize_array(array, target_fields):
        for field in target_fields:
            if field not in array.fields:
                array = ak.with_field(array, None, field)
        return ak.zip({field: array[field] for field in target_fields})
    print("Start harmonizing fields")
    # Apply harmonization
    truth_array = harmonize_array(particle_only, reco_field_order)
    cut_array = harmonize_array(cut_array, reco_field_order)
    print("Done harmonizing")
    df_particle = ak.to_dataframe(particle_only)
    df_reco = ak.to_dataframe(reco_only)
    df_matched = ak.to_dataframe(cut_array)
    print("Converted to pandas - Start concatenating")
    # Concatenate
    df_final = pd.concat([df_particle, df_reco, df_matched], ignore_index=True)

    #final_array = ak.concatenate([particle_only, reco_only, cut_array], axis=0)
    
    return df_final

In [ ]:
def avoid_double_counting(array):
    ids = array["event"]

    # Use numpy to find unique ones and return indices of the first occurrences
    _, unique_index = np.unique(ak.to_numpy(ids), return_index=True)

    # Sort indices to preserve original order
    unique_index = np.sort(unique_index)

    # Select only unique entries
    unique_awkward = array[unique_index]
    return unique_awkward

#truth_array = avoid_double_counting(truth_array)
#herwig_array = avoid_double_counting(herwig_array)
#pythia_array_reco_cuts = avoid_double_counting(pythia_array_reco_cuts)
#pythia_array_matched_cuts = avoid_double_counting(pythia_array_matched_cuts)

In [ ]:
def fix_pass_particle(df):
    """
    Ensure consistency: pass_reco can only be True if key reco observables are non-zero.
    Key reco observables: ptl1, ptj1, ptb1
    """
    key_reco = ['truth_ptj1', 'truth_ptj2']
    
    # Create a mask: pass_reco should be True only if all key_reco > 0
    mask_valid = (df[key_reco] != 0.0).all(axis=1)
    
    # Set pass_reco to False where key observables are zero
    df.loc[df['pass_particle'] & ~mask_valid, 'pass_particle'] = False
    
    return df

In [ ]:
#reco_herwig = '/scratch/mjosef/OMNIFOLD_Tutorial/datasets/WbWb_files/bulk_region/ttbar_enhanced_Herwig71_fast.Nominal.root'
#truth_pythia = '/scratch/mjosef/OMNIFOLD_Tutorial/datasets/WbWb_files/bulk_region/truth_ttbar_enhanced.Nominal.root'
#reco_pythia = '/scratch/mjosef/OMNIFOLD_Tutorial/datasets/WbWb_files/bulk_region/ttbar_enhanced.Nominal.root'
#
#
#herwig_file = uproot.open(reco_herwig)
#herwig_tree = herwig_file['NOSYS/4j_incl']
#
#pythia_truth_file = uproot.open(truth_pythia)
#pythia_truth_tree = pythia_truth_file['NOSYS/physics']
#
#pythia_reco_file = uproot.open(reco_pythia)
#pythia_reco_tree = pythia_reco_file['NOSYS/4j_incl']
#pythia_matched_tree = pythia_reco_file['NOSYS/matched_4j_incl']

#herwig_array = herwig_tree.arrays()
#pythia_array = pythia_tree.arrays()


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pickle
import h5py

# --------------------------------------------------------
# --- USER CONFIGURATION ---
# --------------------------------------------------------
weights_folder = '/scratch/mjosef/Unfolding/omnifold/weights_real-data'
omnifold_name  = "WWbb_transformer_real-data"   # whatever prefix you used
n_iterations   = 4
n_ensembles    = 1                  # or 3 if you used ensemble


# luminosity & normalization
luminosity = 3244.54 + 33402.2 + 44630.6 + 58791.6
SF = (1 / luminosity) * (1 / train_fraction)
# --------------------------------------------------------


def load_unfolded_weights(iteration, stepn, ensemble=0):
    """
    Load OmniFold weights for a given iteration and step.
    """
    model_name = f"{weights_folder}/OmniFold_{name}_iter{iteration}_step{stepn}"
    if n_ensembles > 1:
        model_name += f"_ensemble{ensemble}"
    model_name += ".weights.h5"

    if not os.path.exists(model_name):
        print(f"❌ Missing: {model_name}")
        return None

    print(f"✅ Loaded weights: {model_name}")

    # Example: load reweighted model and evaluate on generator-level MC
    my_model = omnifold.model2
    my_model.load_weights(model_name)
    unfolded_weights = my_model.predict(pythia_truth_test, batch_size=1000).flatten()

    return unfolded_weights


def plot_iteration(iteration, unfolded_weights):
    """
    Reproduce your TUnfold comparison plot for a given iteration.
    """
    hist = TUnfold_incl_file['unfolding_ptl1_NOSYS']
    rel_ptl1_up = TUnfold_incl_file['unfolding_error_ptl1_direct_STAT_DATA__1up;1']
    rel_ptl1_down = TUnfold_incl_file['unfolding_error_ptl1_direct_STAT_DATA__1down;1']

    values = hist.values()
    edges = hist.axis().edges()

    bin_widths = np.diff(edges)
    rel_unc_up_tunfold = rel_ptl1_up.values()
    rel_unc_down_tunfold = rel_ptl1_down.values()

    # --- OmniFold histograms ---
    weights_omnifold = (unfolded_weights * pythia_weights_test)[pythia_pass_gen_test & flags_mc] * SF
    weights_omnifold2 = weights_omnifold**2
    values_omnifold = pythia_truth_test[:, 0, 0][pythia_pass_gen_test & flags_mc]

    counts2, _ = np.histogram(values_omnifold, bins=edges, weights=weights_omnifold)
    counts2_density = counts2 / bin_widths

    sum_w2, _ = np.histogram(values_omnifold, bins=edges, weights=weights_omnifold2)
    with np.errstate(divide='ignore', invalid='ignore'):
        rel_unc_omnifold = np.sqrt(sum_w2) / counts2
        rel_unc_omnifold[~np.isfinite(rel_unc_omnifold)] = 0

    # --- Pythia reference ---
    counts1, _, _ = plt.hist(
        pythia_loader.gen[:, 0, 0][pythia_loader.pass_gen],
        weights=pythia_loader.weight[pythia_loader.pass_gen] * SF,
        bins=edges,
        histtype='step'
    )
    plt.clf()  # clear dummy hist

    counts1_density = counts1 / bin_widths
    values_density = values / bin_widths
    abs_unc_up_tunfold = rel_unc_up_tunfold * values_density
    abs_unc_down_tunfold = rel_unc_down_tunfold * values_density
    bin_centers = 0.5 * (edges[:-1] + edges[1:])

    # --- Figure ---
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 8),
                                   gridspec_kw={'height_ratios': [3, 1]},
                                   sharex=True)

    ax1.step(edges[:-1], counts1_density, where="post", label=r"Pythia8 $t\bar{t}$")
    ax1.step(edges[:-1], counts2_density, where="post", label=f"OmniFold Iter {iteration+1}")
    yerr = np.vstack((abs_unc_down_tunfold, abs_unc_up_tunfold))
    ax1.errorbar(bin_centers[:-1], values_density[:-1], yerr=yerr[:, :-1],
                 fmt='o', color='black', capsize=3, markersize=4, label="TUnfold (stat. unc.)")
    ax1.set_yscale('log')
    ax1.set_ylabel("cross-section [pb/GeV]")
    ax1.legend()
    ax1.set_title(f"OmniFold vs TUnfold (Iteration {iteration+1})")

    # --- Ratio ---
    ratio2 = np.divide(counts2_density, values_density, out=np.zeros_like(counts2_density), where=values_density != 0)
    ratio3 = np.ones_like(values_density)
    upper_omnifold = ratio2 * (1 + rel_unc_omnifold)
    lower_omnifold = ratio2 * (1 - rel_unc_omnifold)
    err_up_tunfold = ratio3 * rel_ptl1_up.values()
    err_down_tunfold = ratio3 * rel_ptl1_down.values()
    yerr_tunfold = np.vstack((err_down_tunfold, err_up_tunfold))

    ax2.step(edges[:-1], ratio2, where="post", color="orange", label="Omnifold / TUnfold")
    ax2.errorbar(bin_centers[:-1], ratio3[:-1], yerr=yerr_tunfold[:, :-1],
                 fmt='o', color='black', capsize=3, label='TUnfold stat. unc.')
    ax2.fill_between(edges[:-1], lower_omnifold, upper_omnifold, step='post', alpha=0.2,
                     color='orange', label='Omnifold stat. unc.')
    ax2.set_xlabel("Lepton-pT [GeV]")
    ax2.set_ylabel("Ratio to TUnfold")
    ax2.set_xlim(30, 700)
    ax2.set_ylim(0.5, 1.5)
    ax2.legend(fontsize=8)
    ax2.grid()

    plt.tight_layout()
    plt.savefig(f"{weights_folder}/plot_iter{iteration+1}.png", dpi=150)
    plt.close()
    print(f"📈 Saved plot for iteration {iteration+1}")
    

# --------------------------------------------------------
# MAIN LOOP
# --------------------------------------------------------
for i in range(n_iterations):
    unfolded_weights = load_unfolded_weights(i, stepn=2)  # use step2 model output
    if unfolded_weights is not None:
        plot_iteration(i, unfolded_weights)

In [ ]:
def MC_data_shaper(df):

    particle_names = ['l1', 'b1', 'b2', 'b3', 'b4', 'j1', 'j2', 'j3', 'j4', 'j5', 'j6']

    # Build reco and truth arrays
    reco_pts  = [df[f'pt{p}']   for p in particle_names]
    reco_etas = [df[f'eta{p}']  for p in particle_names]
    reco_phis = [df[f'phi{p}']  for p in particle_names]
    
    # Masses: lepton = 0, b-jets = mb1–4, jets = mj1–6
    reco_masses = [np.zeros_like(df['ptl1'])] + \
                  [df[f'mb{i}'] for i in range(1, 5)] + \
                  [df[f'mj{i}'] for i in range(1, 7)]
    # Stack into (12 particles, n_events, 4)
    reco_features = np.stack([reco_pts, reco_etas, reco_phis, reco_masses], axis=-1)
    reco_features = reco_features.transpose(1, 0, 2)  # → (n_events, 12, 4)

    # Neutrino (reco): met, eta=0, phi=metphi, mass=0
    met_pt = df['met']
    met_phi = df['metphi']
    zeros = np.zeros_like(met_pt)
    neutrino_reco = np.stack([met_pt, zeros, met_phi, zeros], axis=-1)  # (n_events, 4)

    # Append neutrino
    reco_features = np.concatenate([reco_features, neutrino_reco[:, None, :]], axis=1)  # (n_events, 12, 4)
    truth_pts  = [df[f'truth_pt{p}']   for p in particle_names]
    truth_etas = [df[f'truth_eta{p}']  for p in particle_names]
    truth_phis = [df[f'truth_phi{p}']  for p in particle_names]
    
    truth_masses = [np.zeros_like(df['truth_ptl1'])] + \
                   [df[f'truth_mb{i}'] for i in range(1, 5)] + \
                   [df[f'truth_mj{i}'] for i in range(1, 7)]

    truth_features = np.stack([truth_pts, truth_etas, truth_phis, truth_masses], axis=-1)
    truth_features = truth_features.transpose(1, 0, 2)

    # Neutrino (truth): use 'truth_met' and 'truth_met_phi'
    truth_met_pt = df['truth_met']
    truth_met_phi = df['truth_metphi']
    zeros_truth = np.zeros_like(truth_met_pt)
    neutrino_truth = np.stack([truth_met_pt, zeros_truth, truth_met_phi, zeros_truth], axis=-1)

    truth_features = np.concatenate([truth_features, neutrino_truth[:, None, :]], axis=1)
    
    return reco_features, truth_features

In [ ]:
def add_pass_flags(df: pd.DataFrame) -> pd.DataFrame:
    """
    Adds 'pass_reco' and 'pass_particle' columns based on 'pass_matched'.
    
    Rules:
    - pass_matched == 0 -> pass_reco = 1, pass_particle = 0
    - pass_matched == 2 -> pass_reco = 0, pass_particle = 1
    - pass_matched == 1 -> pass_reco = 1, pass_particle = 1
    """
    df = df.copy()
    
    df["pass_reco"] = 0
    df["pass_particle"] = 0

    # Apply the rules
    df.loc[df["pass_matched"] == 0, "pass_reco"] = 1
    df.loc[df["pass_matched"] == 0, "pass_particle"] = 0

    df.loc[df["pass_matched"] == 2, "pass_reco"] = 0
    df.loc[df["pass_matched"] == 2, "pass_particle"] = 1

    df.loc[df["pass_matched"] == 1, "pass_reco"] = 1
    df.loc[df["pass_matched"] == 1, "pass_particle"] = 1

    return df